In [10]:
#数据包导入
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
import matplotlib.pyplot as plt

#数据集导入
#训练集
train_data = KMNIST(root="data", train=True, download=True, transform=ToTensor())
#测试集
test_data = KMNIST(root= "data", train=False, download=True, transform=ToTensor())


In [11]:
#超参数设计
epochs = 10
Lr = [0.1, 0.01]
Batch_size = [64, 128]

In [12]:
Model = nn.Sequential(
    nn.Linear(784,512),
    nn.Sigmoid(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,10)
)

In [15]:
#对模型进行训练
# for i in range(epochs):
for lr in Lr:
    for bs in Batch_size:
        #模型设计
        Model = nn.Sequential(
            nn.Linear(784,512),
            nn.Sigmoid(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
        #处理训练集
        train_dl = DataLoader(train_data,batch_size=bs, shuffle=True )
        #损失函数设计
        loss_fn = nn.CrossEntropyLoss()
        #优化器设置
        optimizer = torch.optim.AdamW(Model.parameters(),lr=lr)
        for i in range(epochs):
            correct = 0
            total = 0
            for x,y in train_dl:
                perd = Model(x.reshape(-1,784))
                loss_val = loss_fn(perd,y)
                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()
                _,perd1 = torch.max(perd,1)
                total += y.size(0)
                correct += (perd1 == y).sum().item()
            if i%5 == 0:
                acc = correct/total*100
                print(f"Epochs :{i},lr: {lr},batch_size: {bs},loss: {loss_val},acc: {acc:>.4f}%")
                correct = 0
                total = 0

Epochs :0,lr: 0.1,batch_size: 64,loss: 1.2935144901275635,acc: 49.3500%
Epochs :5,lr: 0.1,batch_size: 64,loss: 2.2779979705810547,acc: 10.2033%
Epochs :0,lr: 0.1,batch_size: 128,loss: 2.3015520572662354,acc: 10.0917%
Epochs :5,lr: 0.1,batch_size: 128,loss: 2.310360908508301,acc: 9.8217%
Epochs :0,lr: 0.01,batch_size: 64,loss: 0.25558722019195557,acc: 85.0300%
Epochs :5,lr: 0.01,batch_size: 64,loss: 0.5244665741920471,acc: 94.7517%
Epochs :0,lr: 0.01,batch_size: 128,loss: 0.220633402466774,acc: 81.8033%
Epochs :5,lr: 0.01,batch_size: 128,loss: 0.2737196981906891,acc: 95.9383%


In [14]:
#在进行模型验证
#处理测试集
test_dl = DataLoader(test_data,batch_size=64)
#总样本数初始化
total = 0
#预测正确的总样本数
correct = 0
#因为进入推理模式不进行梯度更新
with torch.no_grad():
    for data,target in test_dl:
        output=Model(data.reshape(-1,784))
        #选取预测可能性最大的一个类
        _,perd = torch.max(output,1)
        total += target.size(0)
        correct += (perd == target).sum().item()
print(f"预测准确率为{correct/total*100:>.4f}%")

预测准确率为88.1700%
